In [3]:


import json
import pickle 
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

import random

2024-04-12 16:00:35.567791: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 16:00:35.570662: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-12 16:00:35.609500: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 16:00:36.300424: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
# load intent.json
data_file = open('Intent.json').read()
intents = json.loads(data_file)

In [8]:
# pre-process Data
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

# Download nltk resources
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

#initialize 
words = []
classes = []
document = []
ignore_words = ['?', '!']

# create a lemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

# tokenize ,lemmatize and stopwords removal

tokenized_data = []
for intent in intents['intents']:
    for text in intent['text']:
        # tokenize each word
        tokens = nltk.word_tokenize(text.lower()) # tokenize and convert to lowercase
        words.extend(tokens)
        # add document in corpus
        document.append((tokens,intent['intent']))
        lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens] # lemmatize tokens
        filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words] # Remove stop words
        tokenized_data.append(filtered_tokens)

# Remove duplicate words 
for i in range(len(tokenized_data)):
    tokenized_data[i] = sorted(list(set(tokenized_data[i])))

print(tokenized_data)
    

[nltk_data] Downloading package punkt to /home/asmita/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/asmita/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/asmita/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['hi'], ['hi'], ['hola'], ['hello'], ['hello'], ['hya'], ['hya'], ['adam', 'user'], ['adam'], ['adam'], ['adam'], ['bella', 'user'], ['bella'], ['bella'], ['bella'], ['?'], ['?', 'hi'], ['?', 'hello'], ['?', 'hola'], ['?'], ['?', 'hope', 'well'], ['?', 'hello', 'hope', 'well'], ['!', 'adam', 'good', 'thanks', 'user'], ['!', 'adam', 'good', 'thanks'], ['!', 'adam', 'good', 'thanks'], ['!', 'adam', 'good', 'thanks'], ['!', 'bella', 'great', 'thanks', 'user'], ['!', 'bella', 'great', 'thanks'], ['!', 'bella', 'great', 'thanks'], ['!', 'bella', 'great', 'thanks'], ['?', 'name'], ['?', 'call'], ['?', 'think'], ['?', 'think'], ['?', 'talking'], ['?', 'call', 'name'], ['name', 'tell'], ['?', 'name'], ['?', 'call', 'could'], ['?', 'call'], ['?', 'call', 'friend'], ['?'], ['?', 'name', 'tell'], ['?', 'name', 'real'], ['?', 'name', 'please', 'real'], ["'s", '?', 'name', 'real'], ['?', 'name', 'real', 'tell'], ['?', 'name', 'real'], ['?', 'name', 'please', 'real'], ['?', 'name', 'please', 'real'

In [9]:
# training and testing data
training = []
output_empty = 0 * len(tokenized_data)

for doc in documents:
    bag = []
    patter_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower() for word in pattern_words )]

    for w in words:
        bag.append(1) if w in patter_words else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag,output_row])

# shuffle and convert it to numpy array
random.shuffle(training)
training = np.array(training)


# separate features and lables
train_x = list(training[:,0])
train_y = list(training[:,1])

                     

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
#build the model

model = Sequential()

model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

# Save the model

model.save('chatbot_model.h5')